In [1]:
import requests
import bs4 as bs
import lxml
from pprint import pprint

httpaddr = "https://www.infobae.com"
requests_ret = requests.get(httpaddr)
print(requests_ret, end = ' || ')
soup = bs.BeautifulSoup(requests_ret.text, 'lxml')
# discard <script> and <style> tags
for discard_tag in ("script", "style"):
    for t in soup.find_all(discard_tag): t.extract()
tags_tuple = ('article', 'section', 'header') + tuple(('h' + str(i) for i in range(1, 7)))
tags = {tag_str : soup.find_all(tag_str) for tag_str in tags_tuple}
for i, tag in enumerate(tags):
    print(f"{tag}s : {len(tags[tag])}", sep = '', end = '\n' if i == len(tags) - 1 else ' / ')



<Response [200]> || articles : 0 / sections : 0 / headers : 0 / h1s : 0 / h2s : 102 / h3s : 15 / h4s : 0 / h5s : 0 / h6s : 0


In [2]:
# Some utility functions - print_tag_subtree - depth
def print_tag_subtree(tag, prefix):
    stack = []
    stack.extend(reversed([ch for ch in tag.children if isinstance(ch, bs.element.Tag)]))
    text_payload = tag.get_text()
    print(prefix + tag.name + ': ' + 
        ('[' + str(tags['h2'].index(tag)) + '] : ' +
        tag.get_text() if tag.name == 'h2' else ''))

    while stack:
        print_tag_subtree(stack.pop(), ' | ' + prefix)

def depth(tag):
    tag_depth = 0
    while not isinstance(tag, bs.BeautifulSoup):
        tag_depth += 1
        tag = tag.parent
    return tag_depth

In [3]:
# Check for <h2> nesting
for i, h2_i in enumerate(tags['h2']):
    for j, h2_j in enumerate(tags['h2']):
        if h2_j == h2_i:
            continue
        if h2_j in list(h2_i.find_all('h2')):
            print(f'<h2> [{j}] in <h2> [{i}]')
# 

In [4]:
# Clusters <h2>s according to the nearest upper tag containing more than one <h2>
# WARNING: This is brittle under some conditions like "Live streaming" & "News Flash"
# TOOD: ... exploit such brittleness to:
# ... 1) Detect "Live Streams" and "Newsflashes"
# -- this could be generalized and canned into a function -- 
clusters_list = []
clusters_dict = {}
for i, h2 in enumerate(tags['h2']):
    tag = h2.parent
    while len(list(tag.find_all('h2'))) == 1:
        tag = tag.parent
    h2_cluster = list(tag.find_all('h2'))
    if list(tag.find_all('h2')) in clusters_list: 
        continue
    else:
        clusters_list.append(list(tag.find_all('h2')))
        clusters_dict[tag] = list(tag.find_all('h2'))

    
#   print(i, '>', len(list(tag.find_all('h2'))))
#   for t in tag.find_all('h2'):
#       print('\t', t.get_text())
    # at this point, h2 is in a given cluster
print('Number of Clusters (list): ', len(clusters_list), sep = '')
print('Number of Clusters (dict): ', len(clusters_dict), sep = '')
print('-' * len('Number of Clusters (dict): ' + str(len(clusters_dict))))
for i, cluster in enumerate(clusters_list):
    if i == 0: continue
    print(f'Cluster Number {i} -- Number of members on the cluster {len(cluster)}')
    print('-' * len(f'Cluster Number {i} -- Number of members on the cluster {len(cluster)}'))
    for j, member in enumerate(cluster):
        print(f"|\tOrder within cluster {j} -- Order within headers {tags['h2'].index(member)}")
        print(f"|\tText :: {member.get_text()}")
        if j + 1 < len(cluster): print("|")
    print("End_Of_Cluster", '-' * 40, '\n')



Number of Clusters (list): 31
Number of Clusters (dict): 31
-----------------------------
Cluster Number 1 -- Number of members on the cluster 3
------------------------------------------------------
|	Order within cluster 0 -- Order within headers 3
|	Text :: En el peronismo descuentan que la militancia se movilizará ante una eventual condena a CFK
|
|	Order within cluster 1 -- Order within headers 4
|	Text :: El inició del dólar soja 2 incrementó las ventas pero en menor medida que en la primera etapa del plan
|
|	Order within cluster 2 -- Order within headers 5
|	Text :: Quién es el sindicalista que amenaza con parar el Estado si la Justicia condena a Cristina Kirchner
End_Of_Cluster ---------------------------------------- 

Cluster Number 2 -- Number of members on the cluster 95
-------------------------------------------------------
|	Order within cluster 0 -- Order within headers 0
|	Text :: Integrantes de los movimientos sociales amenazan con irse del Gobierno en medio de la te

In [5]:
# Show the size of the cluster associated to each <h2>:
for i, h2 in enumerate(tags['h2']):
    cursor = h2.parent
    while len(list(cursor.find_all('h2'))) == 1:
        cursor = cursor.parent
    print(f" <h2> {i:03d} : {len(list(cursor.find_all('h2')))}", end = '\n' if (i + 1) % 6 == 0 else ' |')


 <h2> 000 : 3 | <h2> 001 : 3 | <h2> 002 : 3 | <h2> 003 : 3 | <h2> 004 : 3 | <h2> 005 : 3
 <h2> 006 : 95 | <h2> 007 : 4 | <h2> 008 : 4 | <h2> 009 : 4 | <h2> 010 : 4 | <h2> 011 : 95
 <h2> 012 : 4 | <h2> 013 : 4 | <h2> 014 : 4 | <h2> 015 : 4 | <h2> 016 : 2 | <h2> 017 : 2
 <h2> 018 : 3 | <h2> 019 : 3 | <h2> 020 : 3 | <h2> 021 : 4 | <h2> 022 : 4 | <h2> 023 : 4
 <h2> 024 : 4 | <h2> 025 : 3 | <h2> 026 : 3 | <h2> 027 : 3 | <h2> 028 : 2 | <h2> 029 : 2
 <h2> 030 : 2 | <h2> 031 : 2 | <h2> 032 : 4 | <h2> 033 : 4 | <h2> 034 : 4 | <h2> 035 : 4
 <h2> 036 : 2 | <h2> 037 : 2 | <h2> 038 : 3 | <h2> 039 : 3 | <h2> 040 : 3 | <h2> 041 : 4
 <h2> 042 : 4 | <h2> 043 : 4 | <h2> 044 : 4 | <h2> 045 : 3 | <h2> 046 : 3 | <h2> 047 : 3
 <h2> 048 : 4 | <h2> 049 : 4 | <h2> 050 : 4 | <h2> 051 : 4 | <h2> 052 : 2 | <h2> 053 : 2
 <h2> 054 : 95 | <h2> 055 : 3 | <h2> 056 : 3 | <h2> 057 : 3 | <h2> 058 : 95 | <h2> 059 : 2
 <h2> 060 : 2 | <h2> 061 : 4 | <h2> 062 : 4 | <h2> 063 : 4 | <h2> 064 : 4 | <h2> 065 : 95
 <h2> 066 : 4 | 

In [6]:
# for i, h2 in enumerate(tags['h2']):
#     print(f'{i:3d}')
#     print(h2.parent.prettify())
#     print()  
opinion_h2s = []
for i, h2 in enumerate(tags['h2']):
    if 'opi_hl' in h2.get('class'):
        print(f'<h2>[{i}]'); print('-' * len(f'<h2>[{i}]'))
        print(h2.parent.parent.prettify())
        print()

TypeError: argument of type 'NoneType' is not iterable

In [ ]:
for i, h2 in enumerate(tags['h2']):
    print(f'<h2>[{i:3d}]>', h2.get_text())

<h2>[  0]> Horacio Rodríguez Larreta rechazó cambios en el sistema electoral en 2023: “Está mal, es querer hacer trampa”
<h2>[  1]> Cansado de intentar producir autos eléctricos en el país, se va a Brasil: “Te desaniman día a día”
<h2>[  2]> Caso García Belsunce: las palabras de Pachelo ante los jueces tras el pedido de prisión perpetua
<h2>[  3]> “La próxima te rompo la cara”: fuerte cruce entre Patricia Bullrich y un hombre de confianza de Larreta
<h2>[  4]> “No voy a pedir disculpas”: Patricia Bullrich habló de su ataque a Felipe de Miguel
<h2>[  5]> Laura Blaquier: “El país es una ruleta rusa, cada día hay familias destruidas por la inseguridad”
<h2>[  6]> Los aumentos que llegan en noviembre: tarifas de gas, luz, colegios privados y alquileres
<h2>[  7]> Son 24 las argentinas camino a ser santas: las causas de las mujeres pueden esperar
<h2>[  8]> Ni la "grieta" es un invento argentino ni la esclavitud terminó: leé gratis el libro que explica por qué
<h2>[  9]> Un influencer se co

In [ ]:
for i, h3 in enumerate(tags['h3']):
    print(f'<h3>[{i}]'); print(len(f'<h3>[{i}]') * '-')
    print(h3.parent.prettify())
    print('-' * 80)

<h3>[0]
-------
<a class="headline-link" href="/politica/2022/11/01/no-voy-a-pedir-a-disculpas-patricia-bullrich-hablo-de-su-ataque-a-felipe-de-miguel/">
 <h3 class="headline-feed-column">
  <span>
   “No voy a pedir disculpas”: Patricia Bullrich habló de su ataque a Felipe Miguel
  </span>
 </h3>
</a>
--------------------------------------------------------------------------------
<h3>[1]
-------
<a class="headline-link feed-column-card" href="/sociedad/2022/11/01/laura-blaquier-el-pais-es-una-ruleta-rusa-todos-los-dias-hay-familias-destruidas-por-la-inseguridad/">
 <h3 class="headline-feed-column">
  <span>
   Laura Blaquier: “El país es una ruleta rusa, todos los días hay familias destruidas por la inseguridad”
  </span>
 </h3>
</a>

--------------------------------------------------------------------------------
<h3>[2]
-------
<a class="headline-link feed-column-card" href="/economia/campo/2022/11/01/altisima-presion-fiscal-el-estado-se-lleva-casi-la-mitad-de-los-dolares-producido